In [1]:
### Openlaw - 硬爬模块

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent
import json
import time
import random
from urllib.parse import quote
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from PIL import Image,ImageEnhance
import hashlib
from collections import Counter

In [ ]:
### API查询模块，最终得到的是一个含有案件号的excel文件

def extract(html_list):
    list_cases = list(map(lambda x:x.text,html_list))
    list_cases_2 = list(map(lambda x:eval(x)["judgements"],list_cases))
    cases_3 = list(np.array(list_cases_2).flatten())
    list_case = []
    for index in range(0,10):
        for item in cases_3[index]:
            elem = item["judgement"]
            list_case.append(elem)
    return list_case

class openlaw():
    def __init__(self):
        self.AppKey = "65d54678b0ec414baf31386fa799cbca"
        self.AppSecret = "90a0af2cb7864ab28e160a89a9398436"
        self.Nonce = str("12sddrobc343fghasf5htl56")
        self.Curtime = str(int(time.time() * 1000))
    
    def openlaw_api_headers(self):
        self.Combined = self.AppSecret + self.Nonce + self.Curtime
        sha1 = hashlib.sha1(self.Combined.encode("utf-8"))
        self.CheckSum = sha1.hexdigest()
        self.headers = {"AppKey":self.AppKey,
                   "Nonce":self.Nonce,
                   "CurTime":self.Curtime,
                   "CheckSum":self.CheckSum}
        return self.headers

    def openlaw_api_search(self):
        target = "http://develop.openlaw.cn/judgement/search"
        keyword = input("请输入需要查询的案例关键词：")
        url = target + "?keyword=" + keyword
        ### 以下为选填参数
        """
        请求参数说明
        参数	参数说明
        keyword	搜索关键字
        court	法院名称
        judgeDateYear	判决年份
        """
        switch = True
        while switch == True:
            court,year = None,None
            paramater_none = input("是否需要其他设定参数（输入y/n）")
            if paramater_none == "y":
                pass
            else:
                break
            paramater = input("请输入还需要限定的参数")
            if paramater == "法院名称":
                court = input("哪个法院？")
                url = url + "&court=" + str(court)
            if paramater == "判决年份":
                year = input("哪一年?")
                url = url + "&judgeDateYear=" + str(year)
            paramater_none = input("还需要设定参数吗？（输入y/n）")
            if paramater_none == "n":
                switch = False
            else:
                switch = True
        headers = self.openlaw_api_headers()
        html = requests.get(url,headers = headers)
        return html
    
    def openlaw_api_search_v2(self,keyword):
        #### 设定法院进行爬取
        target = "http://develop.openlaw.cn/judgement/search"
        list_urls = []
        url = target + "?keyword=" + keyword
        with open("courts.txt","r",encoding = "utf-8") as f:
            court_list = eval(f.read())
        #### 选择你需要的法院
        for court in court_list[0:10]:
            url_single = url + "&court=" + str(court)
            list_urls.append(url_single)
        headers = self.openlaw_api_headers()
        html_list = list(map(lambda x:requests.get(x,headers = headers),list_urls))
        list_case = extract(html_list)
        return list_case
        
    def openlaw_api_court(self):
        target = "http://develop.openlaw.cn/judgement/court"
        dicts = {}
        keycourt = input("请输入需要查询的法院：")
        headers = self.openlaw_api_headers()
        dicts["court"] = keycourt
        html = requests.get(target,headers = headers,data = data)
        return html
        
    def openlaw_api_analysis(self):
        target = "http://develop.openlaw.cn/analytic/judgement"
        dicts = {}
        keycourt = input("请输入需要查询的关键字：")
        dicts["keyword"] = keyword
        html = requests.get(target,headers = headers,data = data)
        return html
        
o1 = openlaw()
#html_list = o1.openlaw_api_search()
#html_list = o1.openlaw_api_search_v2("合同")
#pd.DataFrame(html_list).to_excel("合同100_v2.xlsx")

In [ ]:
## from PIL import Image,ImageEnhance
## 强行爬取openlaw上的法条
## 可以做一点大的改进，晚上最后来做

from bs4 import BeautifulSoup
import random

def login(dicts,login_entrance,account,password):
    driver = webdriver.Chrome("chromedriver.exe")
    driver.get(login_entrance)
    elem_account = driver.find_element_by_id("username")
    elem_account.send_keys(login_account)
    elem_password = driver.find_element_by_id("password")
    elem_password.send_keys(login_password)
    elem_verification = driver.find_element_by_id("code")
    code = input("验证码是多少:") # 手工打码
    elem_verification.send_keys(code)
    driver.find_element_by_id("submit").click()
    response = driver.page_source
    html_soup = BeautifulSoup(response,'lxml')
    return driver

def rounds(dicts,urls,login_entrance,login_account,login_password):
    driver = login(dicts,login_entrance,login_account,login_password)
    print(driver)
    list_soup = []
    index = 0 
    for url in urls[0:100]:
        driver.get(url)
        try:
            element = WebDriverWait(driver,10).until(
                EC.presence_of_element_located((By.CLASS_NAME,"entry-title"))
            )
            time.sleep(5)
        except Exception as e:
            time.sleep(5)
            pass
        html_soup = BeautifulSoup(driver.page_source,'lxml')
        list_soup.append(html_soup)
        index += 1
        if index % 10 == 0:
            time.sleep(10)
        if index % 50 == 0:
            time.sleep(50)
        if index % 100 == 0:
            pass
    return list_soup
    
def main():
    dicts = {"account":["hongruzyj@hotmail.com","hongruzyj@qq.com"],"password":["945180zyj","945180zyj"]}
    login_account = dicts["account"][0]
    login_password = dicts["password"][0]
    table = pd.read_excel("合同100.xlsx")
    url_bash = "http://openlaw.cn/judgement/"
    urls = list(map(lambda x:url_bash + x,table["id"]))
    list_soup = rounds(dicts,urls,"http://openlaw.cn/login.jsp",login_account,login_password)
    
## 启动入口
if __name__ == "__main__":
    #main()

In [ ]:
### 清洗得到的结果
list_corpus = []
def get_soup_text(html_soup):
    list_text_temp = []
    try:
        all_text_soup = html_soup.find_all("div",class_ = "part")
        for para in all_text_soup:
            text = re.sub("\n","",para.get_text())
            text2 = re.sub(" ","",text)
            text3 = re.sub(" ","",text2)
            list_text_temp.append(text3)
        return list_text_temp
    except Exception as e:
        return("None")

list_corpus = list(map(get_soup_text,list_soup))
list_corpus_text = list(map(lambda x:"".join(x),list_corpus))
list_corpus_text
table_corpus_with_id = pd.DataFrame([list_corpus_text,list(table['id'])]).T
table_corpus_with_id.columns = ["文本","文本id"]
table_final_original = table_corpus_with_id[table_corpus_with_id["文本"].str.len() > 50]
table_final_original.index = range(len(table_final_original))
table_final_original.to_excel("hetong_wenben.xlsx")